### Init Environment

In [7]:
print('Installing torchprofile...')
!pip install torchprofile
print('All required packages have been successfully installed!')

Installing torchprofile...
All required packages have been successfully installed!


In [8]:
import copy
import math
import random
import time
from collections import OrderedDict, defaultdict
from typing import Union, List

import numpy as np
import torch
from matplotlib import pyplot as plt
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchprofile import profile_macs
from torchvision.datasets import *
from torchvision.transforms import *
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, TensorDataset


### Data

### Model

In [4]:
class Reshape(nn.Module):
    def __init__(self, shape):
        super(Reshape, self).__init__()
        self.shape = shape

    def forward(self, x):
        return x.view((-1,) + self.shape)

class MyModel(nn.Module):
  def __init__(self) -> None:
    super().__init__()

    layers = []
    counts = defaultdict(int)
    seq_len = 90

    def add(name: str, layer: nn.Module) -> None:
      layers.append((f"{name}{counts[name]}", layer))
      counts[name] += 1

    add("gru", nn.GRU(input_size=6, hidden_size=32, batch_first=True))
    add("relu", nn.ReLU(True))
    add("gru", nn.GRU(input_size=32, hidden_size=32, batch_first=True))
    add("relu", nn.ReLU(True))
    add("reshape", Reshape((-1, 32, seq_len)))  
    add("conv", nn.Conv1d(32, 64, 2, stride=2))
    add("relu", nn.ReLU())
    add("pool", nn.MaxPool1d(4, padding='same'))
    add("conv", nn.Conv1d(64, 128, 2, stride=1))
    add("relu", nn.ReLU())
    add("adaptivePoll",nn.AdaptiveAvgPool1d(1))
    add("bn", nn.BatchNorm1d(128, eps=1e-06))

    self.backbone = nn.Sequential(OrderedDict(layers))
    self.classifier = nn.Linear(128, 4)
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    x = self.backbone(x)
    x = self.classifier(x)
    x = self.softmax(x)

    return x
  
model = MyModel().cuda()
## show model summary
print(model.eval())

NameError: name 'nn' is not defined

### Training